### Code for Link Prediction in Bee-Flower Network (Bipartite Network)

In [269]:
##### Import Needed Libraries
import math
import numpy as np
import random
import networkx as nx
from numpy.linalg import matrix_rank
from networkx.algorithms import bipartite
from sklearn.metrics import roc_auc_score

In [270]:
##### Hyper Parameters
##### Number of time-steps (shown by num_graphs), number of bees as the first partite, number of flowers as the second partite
##### History length
##### Based on the structure of our network, we consider odd time-steps as history if we are intended to predict an odd
##### time-step and even history for predicting an even time-step as the last time-step
num_graphs = 30
num_bees = 64
num_flowers = 16
history = 3
all_nodes = num_bees + num_flowers
teta = 0.3

In [271]:
##### Function to compute low-rank representation of a given matrix
def low_rank_approx(SVD=None, A=None, r=1):
    """
    Computes an r-rank approximation of a matrix
    given the component u, s, and v of it's SVD
    Requires: numpy
    """
    if not SVD:
        SVD = np.linalg.svd(A, full_matrices=False)
    u, s, v = SVD
    Ar = np.zeros((len(u), len(v)))
    for i in range(r):
        Ar += s[i] * np.outer(u.T[i], v[i])
    return Ar

In [272]:
##### Graph
def graph_cons(num_bees, num_flowers, num_graphs):
    bees = [i+1000 for i in range(num_bees)]
    flow = [i for i in range(num_flowers)]

    visits = [[-1 for i in range(num_bees)] for j in range(num_graphs)] 

    c = 0
    g = [nx.Graph() for i in range(num_graphs)]
    for c in range(num_graphs):
        g[c].add_nodes_from(bees, bipartite=0)
        g[c].add_nodes_from(flow, bipartite=1)
    
    for i in range(num_bees):
        temp = random.randint(0,num_flowers-1)
        visits[0][i] = temp
        g[0].add_edge(bees[i], visits[0][i]) 
    
    for i in range(num_bees):
        temp1 = [j for j in range(num_flowers)]
        temp1.remove(visits[0][i])
        temp2 = random.randint(0,num_flowers-2)
        temp3 = temp1[temp2]
        visits[1][i] = temp3
        g[1].add_edge(bees[i], visits[1][i])
    
    for counter in range(num_graphs-2):
        c = counter + 2
        for i in range(num_bees):
            t1 = visits[c-1][i]
            t2 = list(g[c-1].neighbors(t1))
            t2.remove(bees[i])
            control = 0
            if len(t2)>0:
                t3 = []
                for j in t2:
                    t3.append(visits[c-2][j-1000])
                if visits[c-1][i] in t3:
                    t3.remove(visits[c-1][i])
                if len(t3)>0:
                    t4 = random.randint(0, len(t3)-1)
                    t5 = t3[t4]
                    visits[c][i] = t5
                    g[c].add_edge(bees[i], t5)
                else:
                    control = 1
            else:
                control = 1
            if control == 1:
                temp1 = [j for j in range(num_flowers)]
                temp1.remove(visits[c-1][i])
                temp2 = random.randint(0,num_flowers-2)
                temp3 = temp1[temp2]
                visits[c][i] = temp3
                g[c].add_edge(bees[i], temp3)
    return g

In [273]:
##### Summing up temporal information from time-steps in the history ordering by time
##### We use the idea of "Link Prediction on Evolving Data using Matrix and Tensor Factorizations" for time ordered summation
##### x: summation adjacency matrix
##### z: to represent adjacency matrix of time-steps in the history
##### consider t=1 as starting point of the history and T as the last (when goal is to predict T+1)
##### the summation formula will be: x(i, j) = Sum(over t=1 -> T) (1 - teta)^(T - t) * z_t(i, j)
def adj_mat(g, num_graphs, history, all_nodes, num_bees, num_flowers):
    t_final = num_graphs - 1 ### index of the last time-step (predition target)
    history_ind = [] ### to store odd/ even indices
    for i in range(history):
        ind = (i+1)*2
        history_ind.append(t_final-ind)

    adj = [] ### to store adjacency matrices of history
    for i in range(len(history_ind)):
        adj.append(nx.adjacency_matrix(g[history_ind[i]]))
    
    adj_arr = [] ### turn adjacency matrices to array
    for i in adj:
        adj_arr.append(i.toarray())
    
    ##### time-ordered summation
    overal_adj = [[0 for i in range(all_nodes)] for j in range(all_nodes)]
    for i in range(len(history_ind)):
        coef = (1 - teta)**(t_final-1-history_ind[i])
        overal_adj = overal_adj + coef * adj_arr[i]
    return overal_adj

In [274]:
##### Low-rank representation, we only consider up-left part as it includes all the information (bee nodes)
def low_rep(overal_adj, num_bees):    
    bee = overal_adj[:num_bees, num_bees:]
    r = matrix_rank(bee) ### getting rank of the matrix

    u, s, v = np.linalg.svd(bee, full_matrices=False) ### SVD decomposition
    low_bee = low_rank_approx((u, s, v), r = r)
    return low_bee

In [275]:
##### last time-step as test
def test_g(g, num_bees):
    test = nx.adjacency_matrix(g[-1])
    test = test.toarray()
    test = test[:num_bees, num_bees:]
    return test

In [276]:
##### getting prediction based on the maximum value in low-rank representation of the summation matrix
def prediction(num_bees, num_flowers, low_bee):
    pred = [[0 for i in range(num_flowers)] for j in range(num_bees)]
    for i in range(num_bees):
        max_index = np.argmax(low_bee[i])
        pred[i][max_index] = 1
    pred = np.array(pred)
    return pred

In [277]:
def res(test, pred):
    test = test.flatten()
    pred = pred.flatten()
    print(roc_auc_score(test, pred))

In [278]:
num_graphs = 30
num_bees = 64
num_flowers = 16
history = 3
all_nodes = num_bees + num_flowers
teta = 0.3
def link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 16, history = 3, teta = 0.3):
    all_nodes = num_bees + num_flowers
    g = graph_cons(num_bees, num_flowers, num_graphs)
    overal_adj = adj_mat(g, num_graphs, history, all_nodes, num_bees, num_flowers)
    low_bee = low_rep(overal_adj, num_bees)
    test = test_g(g, num_bees)
    pred = prediction(num_bees, num_flowers, low_bee)
    res(test, pred)

In [294]:
link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 4, history = 3, teta = 0.3)

0.9270833333333333


In [295]:
link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 8, history = 3, teta = 0.3)

0.9464285714285714


In [296]:
link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 16, history = 3, teta = 0.3)

0.9583333333333334


In [298]:
link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 32, history = 3, teta = 0.3)

0.9838709677419355


In [299]:
link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 64, history = 3, teta = 0.3)

0.9206349206349206


In [300]:
link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 16, history = 1, teta = 0.3)

0.9666666666666667


In [302]:
link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 16, history = 2, teta = 0.3)

0.9916666666666666


In [303]:
link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 16, history = 3, teta = 0.3)

0.9249999999999999


In [305]:
link_prediction(num_graphs = 30, num_bees = 64, num_flowers = 16, history = 4, teta = 0.3)

0.9583333333333334
